In [ ]:
# 필요한 모듈 설치
!pip3 install -r ./requirement.txt

In [4]:
# 모듈 로드
from sentence_transformers import SentenceTransformer, util
from annoy import AnnoyIndex
import numpy as np

In [29]:
# Annoy Class 구현
class AnnoyNearestNeighbor:
    def __init__(self, vectors, texts, metric='angular', n_trees=10): # angular = 코사인 유사도, euclidean : L2 거리
        self.texts = texts
        self.d = vectors.shape[1]
        self.index = AnnoyIndex(self.d, metric) # Annoy Index 생성

        for i, vector in enumerate(vectors):
            self.index.add_item(i, vector)
        self.index.build(n_trees)
    
    def search(self, query_vector, top_k=1):
        indices, distances = self.index.get_nns_by_vector(query_vector, top_k, include_distances=True)
        # anglular의 거리는 1-코사인 유사도이므로 코사인 유사도를 구하기 위해 1을 제거
        similarities = [1 - (d/2) for d in distances]
        similar_texts = [self.texts[i] for i in indices]
        return similar_texts, similarities

In [34]:
# 테스트 코드 - Vector DB 구축
model = SentenceTransformer("jhgan/ko-sbert-sts")

text_list = [
    "원숭이가 노래를 한다.",
    "배가 바다를 떠나 원대한 여정을 시작했다.",
    "그는 자신이 벌레만도 못한 취급을 받을 것이라곤 생각할 수 없었다."
]
text_vectors = model.encode(text_list, normalize_embeddings=True)
vector_db = AnnoyNearestNeighbor(text_vectors, text_list, metric='angular', n_trees=100)

In [35]:
# 테스트 코드 - VectorDB 테스트
query_text = "배가 바다를 떠나 원대한 여정을 시작함."
query_vector = model.encode(query_text, normalize_embeddings=True)
text, sim = vector_db.search(query_vector, 1)
print(f"유사 문장 : {text[0]}\t유사도 : {sim[0] : .4f}")
print(f"SBERT 유사도 : {float(util.cos_sim(model.encode(text), model.encode(query_text))) : .4f}") # Annoy 유사도와 동일해야함!

유사 문장 : 배가 바다를 떠나 원대한 여정을 시작했다.	유사도 :  0.9154
SBERT 유사도 :  0.9857
